# Projet Maths 3

# Questions théoriques

> 1. Quel théorème du cours nous autorise-t-il à estimer l’espérance conditionnelle par la moyenne empirique de simulations conditionnelles.


> 2. Rappeler la loi conditionnelle du vecteur des composantes de Z correspondant aux points de discrétisation sans observation, connaissant les
valeurs prises par les composantes aux sites d’observation.

> 3. Si Y = (Y1, . . . , Yp) est un vecteur de composantes gaussiennes indépendantes, toutes d’espérance nulle et de variance 1, quelle est la loi du
vecteur Z = m + RY où R est une matrice p × p et m est un vecteur de
taille p ?

> 4. En déduire un algorithme de simulation conditionnelle.

In [2]:
# chargement dee dépendances 

import numpy as np 
import math
import matplotlib.pyplot as plt 

In [5]:
#Discrétisation
A=0
B=500
N=101

#Nombre de points de discrétisation 
Delta = (B-A)/(N-1)
discretization_indexes = np.arange(N) 
discretization = discretization_indexes*Delta 

#Paramètres du modèle
mu=-5
a = 50 
sigma2 = 12

#Données
observation_indexes = [0,20,40,60,80,100]
depth = np.array([0,-4,-12.8,-1,-6.5,0])


In [ ]:
def C(h, a, sig_2):
    return (np.exp(-abs(h)/a)*sig_2)

def distance(h,N):
    M=np.one(N+1)
    for i,k in enumerate